In [ ]:
!pip uninstall -y numpy pandas requests pyarrow opencv-python-headless -q
!pip install -U \
  "numpy" \
  "pandas==1.5.3" \
  "requests==2.28.2" \
  "pyarrow==10.0.1" \
  "scipy==1.10.1" \
  "opencv-python-headless==4.8.0.76" \
  "transformers" \
  "datasets" \
  "evaluate" \
  "nltk==3.8.1" \
  "textattack" \
  "gensim==4.3.2" \
  "farasapy==0.0.11"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.5

In [ ]:



import os
os.environ["WANDB_DISABLED"] = "true"

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

# ============================================
# ✅ Imports
# ============================================
import importlib.util
import requests
import evaluate
from farasa.pos import FarasaPOSTagger
from datasets import Dataset as HFDataset
from nltk.corpus import wordnet as wn

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack import Attack
from textattack.attacker import Attacker
from textattack.attack_args import AttackArgs
from textattack.search_methods import GreedyWordSwapWIR
from textattack.goal_functions.classification import UntargetedClassification
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.constraints.pre_transformation.input_column_modification import InputColumnModification
from textattack.datasets import Dataset
from textattack.transformations.word_swaps.word_swap import WordSwap

# ============================================
# ✅ Load FastText Arabic Embeddings
# ============================================
!wget -q https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz
from gensim.models import KeyedVectors
embedding_model = KeyedVectors.load_word2vec_format("cc.ar.300.vec.gz", binary=False)

# ============================================
# ✅ Farasa POS Initialization
# ============================================
farasa_pos = FarasaPOSTagger(interactive=True)

# ============================================
# ✅ Arabic Hybrid Synonym Swap (FastText + WordNet + Sina)
# ============================================
class ArabicTextFoolerWordSwap(WordSwap):
    def __init__(self, max_candidates=50):
        super().__init__()
        self.max_candidates = max_candidates

    def fetch_sina_synonyms(self, word):
        try:
            response = requests.get(f"https://sina.birzeit.edu/synonyms/api/{word}", timeout=2)
            if response.status_code == 200:
                data = response.json()
                return [item["synonym"] for item in data if item.get("synonym")]
        except Exception:
            pass
        return []

    def get_replacement_words(self, word):
        """Generate synonym candidates from FastText, WordNet, and Sina."""
        candidates = set()

        # FastText
        try:
            neighbors = embedding_model.most_similar(word, topn=self.max_candidates)
            candidates.update([w for w, _ in neighbors])
        except KeyError:
            pass

        # Arabic WordNet
        for syn in wn.synsets(word, lang='arb'):
            for lemma in syn.lemma_names('arb'):
                if lemma != word:
                    candidates.add(lemma)

        # Sina API
        sina_syns = self.fetch_sina_synonyms(word)
        candidates.update(sina_syns)

        return list(candidates)

    def _get_transformations(self, current_text, indices_to_modify=None):
        """Returns transformed texts with modified index tracking."""
        transformed_texts = []
        indices_to_modify = indices_to_modify or range(len(current_text.words))

        for idx in indices_to_modify:
            word = current_text.words[idx]
            replacements = self.get_replacement_words(word)

            for replacement in replacements:
                if replacement == word or not replacement.strip():
                    continue

                new_text = current_text.replace_word_at_index(idx, replacement)
                new_text.attack_attrs["newly_modified_indices"] = {idx}
                transformed_texts.append(new_text)

        return transformed_texts

    def extra_repr(self):
        return f"(max_candidates={self.max_candidates})"

# ============================================
# ✅ Load Arabic Dataset
# ============================================
dataset_path = "/content/arbml_arabic_100_reviews.py"
spec = importlib.util.spec_from_file_location("arbml_dataset", dataset_path)
arbml_dataset = importlib.util.module_from_spec(spec)
spec.loader.exec_module(arbml_dataset)
data = arbml_dataset.data

texts, labels = zip(*data)
full_dataset = HFDataset.from_dict({"text": texts, "label": labels})

# ============================================
# ✅ Tokenizer and Preprocessing
# ============================================
model_name = "asafaya/bert-base-arabic"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

full_dataset = full_dataset.map(tokenize_function, batched=True)
full_dataset = full_dataset.rename_column("label", "labels")
full_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# ============================================
# ✅ Fine-Tune Arabic BERT
# ============================================
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    }

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

# ============================================
# ✅ Wrap Model for Attack
# ============================================
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# ============================================
# ✅ Build Arabic TextFooler-Style Attack
# ============================================
def build_arabic_textfooler_attack(model_wrapper):
    transformation = ArabicTextFoolerWordSwap(max_candidates=50)

    constraints = [
        RepeatModification(),
        StopwordModification(),
        InputColumnModification(
            matching_column_labels=["premise", "hypothesis"],
            columns_to_ignore=["premise"]
        ),
        WordEmbeddingDistance(min_cos_sim=0.5),
        PartOfSpeech(allow_verb_noun_swap=True),

    ]

    goal_function = UntargetedClassification(model_wrapper)
    search_method = GreedyWordSwapWIR(wir_method="delete")
    return Attack(goal_function, constraints, transformation, search_method)

# ============================================
# ✅ Run the Attack
# ============================================
attack_dataset = Dataset(list(zip(texts, labels)))
attack = build_arabic_textfooler_attack(model_wrapper)

attack_args = AttackArgs(
    num_examples=len(attack_dataset),
    log_to_csv="arabic_textfooler_combined_results.csv",
    csv_coloring_style="plain",
    disable_stdout=False,
    random_seed=42,
)

attacker = Attacker(attack, attack_dataset, attack_args)
attacker.attack_dataset()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already

100%|██████████| 241M/241M [07:47<00:00, 516kiB/s]


[2025-08-13 23:12:31,146 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[0 (75%)]] --> [[[FAILED]]]

أحمد الشقيري في كتابه رحلتي مع غاندي. لم يحاول إخفاء حقيقته و لم يسعى لتلميع شخصه أو إسمه بخلاف الكثير من الكتاب. فهو يسرد نقاط ضعفه بكل أريحية مقرونة ببعض السبل التي انتهجها بدوره في محاولة جادة لإصلاح نفسه. بداية الكتاب يتكون حقيقة من كتابين. أو من قسمين. يجمعهما بلاد الهند و الكاتب. فالقسم الأول يسرد حكم و مقولات لرجل من عظماء الهند مهاتما غاندي و تعليق المؤلف عليه و محاولة إسقاط حكمه و تأصيلها بالمواقف و الأحاديث النبوية. و القسم الثاني هو عبارة عن رحلة المؤلف لبلاد الهند و تجربته الخاصة هناك التي أحب أن يشارك القراء بها. في البداية الكتاب يحوي العديد من التأملات و الحكم الراقية التي خلص لها الكاتب بعد رحلته و قراءته لعدة كتب في الحكمة بحسب ما ذكر. و يمكننا أن نلمس فيه نبض الإنسان المتحرق على أمته و نفسه. لكن يجب علينا ألا نغفل بعض النقاط السلبية التي تعيب الكتاب و أزعجتني شخصيا. أولها. هو رداءة الترجمة لحكم غاندي المذكورة في القسم الأول من الكتاب. و ا

[Succeeded / Failed / Skipped / Total] 73 / 25 / 1 / 99:  99%|█████████▉| 99/100 [3:34:52<02:10, 130.23s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (84%)]] --> [[1 (54%)]]

[[نعم]] [[انها]] [[من]] الكتب العادية جدا [[رغم]] طابعها [[المضحك]] الاهم من ذلك انني [[لم]] اتكلف عناء [[شرائها]] او قرائتها فقط سمعتها [[لمدة]] ساعتين او اكثر وانا اؤدي [[بعض]] الاعمال الاخرى احببت جدا تجربة الكتب المسموعة لكن [[الكتاب]] كان [[لابأس]] به

[[حسن]] [[اانها]] [[rida21]] الكتب العادية جدا [[وعلىالرغم]] طابعها [[المدهش]] الاهم من ذلك انني [[تعد]]ْ اتكلف عناء [[وشراءها]] او قرائتها فقط سمعتها [[لمد]]َّ[[ة]] ساعتين او اكثر وانا اؤدي [[لكثير]] الاعمال الاخرى احببت جدا تجربة الكتب المسموعة لكن [[لكتاب]] كان [[جدا]]ً به




[Succeeded / Failed / Skipped / Total] 74 / 25 / 1 / 100: 100%|██████████| 100/100 [3:34:59<00:00, 129.00s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[1 (77%)]] --> [[0 (62%)]]

مع [[مهل]] عشت الحزن والفرح وبالرغم من ان الحزن سيطر على اغلب اجواء الرواية إلا انني استمتعت جدا بقراءتها ... اسلوب الكاتب بسيط وسلس مما جعلني انهيها في جلستين تقريبا ... ومع ان النهاية لم تكن تلك التى اطمح ان تكون [[إلا]] ان الرواية حركت بداخلي اشياء كثيرة

مع [[ماخي]] عشت الحزن والفرح وبالرغم من ان الحزن سيطر على اغلب اجواء الرواية إلا انني استمتعت جدا بقراءتها ... اسلوب الكاتب بسيط وسلس مما جعلني انهيها في جلستين تقريبا ... ومع ان النهاية لم تكن تلك التى اطمح ان تكون [[سوى]] ان الرواية حركت بداخلي اشياء كثيرة



+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 74      |
| Number of failed attacks:     | 25      |
| Number of skipped attacks:    | 1       |
| Original accuracy:            | 99.0%   |
| Accuracy under attack:        | 25.0%